In [16]:
import cv2 as cv
import numpy as np
import random
import pathlib
import shutil
import matplotlib.pyplot as plt
from osgeo import gdal,ogr,osr
from tqdm import tqdm

In [17]:
%env SM_FRAMEWORK=tf.keras
import segmentation_models as sm
import imgaug as ia
from keras_segmentation.predict import predict
import cv2
from matplotlib import pyplot as plt
from matplotlib import cm
from skimage.transform import radon, rescale, rotate
from skimage.morphology import (square, rectangle, diamond, disk, cube,
                                octahedron, ball, octagon, star, binary_closing, closing, opening, skeletonize, erosion, dilation)
from skimage.transform import probabilistic_hough_line
from skimage.feature import canny
from skimage.measure import label, regionprops, regionprops_table
from skimage.io import imread, imsave
import math
import pandas as pd
import numpy as np
import glob
import os
from sklearn.metrics import jaccard_score
from skimage.morphology import (square, rectangle, diamond, disk, cube,
                                octahedron, ball, octagon, star, binary_closing, closing, opening, skeletonize, erosion, dilation)
from skimage.transform import rescale, resize, downscale_local_mean
import time

env: SM_FRAMEWORK=tf.keras


In [18]:
id_mosaico = '179323' 
file_shp_1 = '../../daninhas/ortomosaicos/pre-emergente/' + id_mosaico + '/daninha_1.shp'
file_shp_2 = '../../daninhas/ortomosaicos/pre-emergente/' + id_mosaico + '/daninha_2.shp'
name_mosaico = '../../daninhas/ortomosaicos/pre-emergente/'+ id_mosaico + '/ortomosaico.tif'
#output_dataset = '../../daninhas/ortomosaicos/pre-emergente/' + id_mosaico #Pasta de saida
output_shp = '../../daninhas/ortomosaicos/pre-emergente/' + id_mosaico  + "/output_" + id_mosaico + '.shp'
output_dataset = '../../daninhas/dataset'
tif_generated_1 = '../../daninhas/ortomosaicos/pre-emergente/' + id_mosaico + '/daninha_1.tif' #Tip de saida
tif_generated_2 = '../../daninhas/ortomosaicos/pre-emergente/' + id_mosaico + '/daninha_2.tif'

MAX_PIXELS_DANINHA = 5000

In [19]:
def GetExtent(gt,cols,rows):
    ''' Return list of corner coordinates from a geotransform

        @type gt:   C{tuple/list}
        @param gt: geotransform
        @type cols:   C{int}
        @param cols: number of columns in the dataset
        @type rows:   C{int}
        @param rows: number of rows in the dataset
        @rtype:    C{[float,...,float]}
        @return:   coordinates of each corner
    '''
    ext=[]
    xarr=[0,cols]
    yarr=[0,rows]

    for px in xarr:
        for py in yarr:
            x=gt[0]+(px*gt[1])+(py*gt[2])
            y=gt[3]+(px*gt[4])+(py*gt[5])
            ext.append([x,y])
            print(x,y)
        yarr.reverse()
    return ext

In [20]:
def ReprojectCoords(coords,src_srs,tgt_srs):
    ''' Reproject a list of x,y coordinates.

        @type geom:     C{tuple/list}
        @param geom:    List of [[x,y],...[x,y]] coordinates
        @type src_srs:  C{osr.SpatialReference}
        @param src_srs: OSR SpatialReference object
        @type tgt_srs:  C{osr.SpatialReference}
        @param tgt_srs: OSR SpatialReference object
        @rtype:         C{tuple/list}
        @return:        List of transformed [[x,y],...[x,y]] coordinates
    '''
    trans_coords=[]
    transform = osr.CoordinateTransformation( src_srs, tgt_srs)
    for x,y in coords:
        x,y,z = transform.TransformPoint(x,y)
        trans_coords.append([x,y])
    return trans_coords

In [21]:
ds=gdal.Open(name_mosaico)
gt=ds.GetGeoTransform()
cols = ds.RasterXSize
rows = ds.RasterYSize
ext=GetExtent(gt,cols,rows)

src_srs=osr.SpatialReference()
src_srs.ImportFromWkt(ds.GetProjection())
tgt_srs = src_srs.CloneGeogCS()

geo_ext= ReprojectCoords(ext,src_srs,tgt_srs)

xmin = geo_ext[0][0]
ymax = geo_ext[0][1]
xmax = geo_ext[2][0]
ymin = geo_ext[1][1]

pixel_size_x = gt[1]
pixel_size_y = gt[5]

-45.4861271872156 -12.89707810163661
-45.4861271872156 -12.907532193067741
-45.475512627790565 -12.907532193067741
-45.475512627790565 -12.89707810163661


In [11]:
!gdal_rasterize -burn 255 -ot Byte -tr $pixel_size_x $pixel_size_y -te $xmin $ymin $xmax $ymax $file_shp_1 $tif_generated_1

#!gdal_rasterize -burn 255 -ot Byte -tr $pixel_size_x $pixel_size_y -te $xmin $ymin $xmax $ymax $file_shp_2 $tif_generated_2

0...10...20...30...40...50...60...70...80...90...100 - done.


In [49]:
#daninha_1 = gdal.Open(tif_generated_1)
#daninha_band_1 = daninha_1.GetRasterBand(1)

#daninha_2 = gdal.Open(tif_generated_2)
#daninha_band_2 = daninha_2.GetRasterBand(1)

mosaico = gdal.Open(name_mosaico)

red = mosaico.GetRasterBand(1)
green = mosaico.GetRasterBand(2)
blue = mosaico.GetRasterBand(3)
alpha = mosaico.GetRasterBand(4)

nx = mosaico.RasterXSize
ny = mosaico.RasterYSize
file_path = pathlib.Path(name_mosaico)
out_file = pathlib.Path('/')
out_file = file_path.parent/(id_mosaico + ".shp")
path_temp = file_path.parent/'temp_files'

if path_temp.exists():
    shutil.rmtree(str(path_temp))
    os.mkdir(str(path_temp))
else:
    os.mkdir(str(path_temp))
    
dst_img = gdal.GetDriverByName('GTiff').Create(str(path_temp/'outfile.tif'), nx, ny, 1, gdal.GDT_Byte)
dst_img.SetGeoTransform(mosaico.GetGeoTransform())
srs = osr.SpatialReference()
srs.ImportFromWkt(mosaico.GetProjection())
dst_img.SetProjection(srs.ExportToWkt())

0

In [8]:
def random_func(*, percentage: int) -> int:
    '''
    percentage: Valores entre 0 e 100. Caso value > percentag é retornado true, caso não
    é retornado false
    
    description: Utilizada para definir se a função de pré processamento será utilizada ou
    não. 
    
    return:
    bool
    '''
    _value = random.randint(0, 100)
    if _value < percentage:
        #print(_value)
        return False
    else:
        #print(_value)
        return True

In [47]:
model = BACKBONE = 'vgg16'
model = sm.Linknet(backbone_name=BACKBONE, encoder_weights='imagenet', encoder_freeze=True, classes=1, activation='sigmoid', weights = 'jocival/vgg16_Linknet_Test16.hdf5')

In [ ]:
total_images = 20 # Total de imagens desejadas (treino e validacao)
count_yes = 0 # contador para as imagens com sim
count_no = 0 # contador para as imagens com sim
count_validation = 0
iterator_x = 256# incremento no eixo x
iterator_y = 256 # incremento no eixo y
expected_yes_perc = 1 # porcentagem entre a quantidade de imagens sim e nao desejadas
expected_validation_perc = 0.25 # porcentagem entre a quantidade de imagens de validacao
black_background = 0.5 # porcentagem de fundo preto aceitavel na imagem


for x in range(0,mosaico.RasterXSize,iterator_x):
    for y in range(0,mosaico.RasterYSize,iterator_y):
        if count_yes + count_no >= total_images:
            break
            
        if ((x+iterator_x)>mosaico.RasterXSize) or ((y+iterator_y)>mosaico.RasterYSize):
            continue
            
        #daninha_parcela = daninha_band_1.ReadAsArray(x,y,iterator_x,iterator_y) #+ daninha_band_2.ReadAsArray(x,y,100,100)
        blueparcela = blue.ReadAsArray(x,y,iterator_x,iterator_y)
        greenparcela = green.ReadAsArray(x,y,iterator_x,iterator_y)
        redparcela = red.ReadAsArray(x,y,iterator_x,iterator_y)
        imgparcela = cv.merge((blueparcela,greenparcela,redparcela))
        img = imgparcela / 255

        pr = model.predict(np.array([img]))[0]
        pr = pr[:,:, 0]
        pr[pr >= 0.1] = 255
        pr[pr < 0.5] = 0
        pr = pr.astype('uint8')
        #print(pr)
        #ia.imshow(pr)
        
        if (imgparcela.max()>0) and (imgparcela.min()<255):
            write_image = pr
            #ia.imshow(write_image)
        else:
            #print('parcela fora da imagem, seguindo para a proxima parcela')
            pr[pr>=255]= 0
            write_image = pr
            
        #print(write_image)
        dst_img.GetRasterBand(1).WriteArray(write_image,xoff=x,yoff=y)
        #cv.imwrite(output_dataset + '/validation/sim/daninha_{x}_{y}.png'.format(x=x,y=y), imgparcela)
        #cv.imwrite(output_dataset + '/saida/sim/daninha_{x}_{y}.png'.format(x=x,y=y), daninha_parcela)
        #count_validation += 
        dst_img.FlushCache()
                

In [ ]:
src_band = dst_img.GetRasterBand(1)

dst_layername = 'daninhas'
drv = ogr.GetDriverByName("ESRI Shapefile")
dst_ds = drv.CreateDataSource(str(out_file))
dst_layer = dst_ds.CreateLayer(dst_layername, srs = srs)

gdal.Polygonize( src_band, src_band, dst_layer, -1, [], callback=None )
dst_ds.Destroy()
#os.remove(str(pathlib.Path(args.input).parent/'temp_filter.png'))

In [ ]:
gdal.Polygonize?